In [20]:
import numpy as np
import matplotlib as plt
from hilp import *
from helper import *
from costs import *
from least_squares import *
from ridge_regression import *
from logistic_regression import *
from cross_validation import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
yb, input_data, ids = load_csv_data('data/train.csv')

In [3]:
yb_te, input_data_te, ids_te = load_csv_data('data/test.csv')

In [4]:
np.shape(yb_te)

(568238,)

In [5]:
input_data.shape

(250000, 30)

In [6]:
cleaned_columns = [c for c in input_data.T if (c==-999).sum()/len(c) < 0.2]
print(np.shape(cleaned_columns))
for c in cleaned_columns:
    numb_of_nan = (c==-999).sum()
    median = np.median(list(filter(lambda x : x!= -999, c)))
    c[c == -999] = median

(20, 250000)


In [7]:
cleaned_columns_te = [c for c in input_data_te.T if (c==-999).sum()/len(c) < 0.2]
print(np.shape(cleaned_columns_te))
for c in cleaned_columns_te:
    numb_of_nan = (c==-999).sum()
    median = np.median(list(filter(lambda x : x!= -999, c)))
    c[c == -999] = median

(20, 568238)


In [8]:
def create_cor_matrix(cleaned_columns):
    cor_matrix = np.corrcoef(cleaned_columns)
    upper_tri = np.triu(cor_matrix)
    for i in range(upper_tri.shape[0]):
        upper_tri[i][i]= 0
    return list(zip(*np.where(upper_tri > 0.6)))

In [9]:
def count(zipped):
    c = [item for z in zipped for item in z]
    return [(x,c.count(x)) for x in set(c)]

In [10]:
zipped = create_cor_matrix(cleaned_columns)
while len(zipped) > 0 :
    biggest = (-1,-1)
    cnt = count(zipped)
    print(cnt)
    for c in cnt:
        if c[1] > biggest[1]:
            biggest = c
    del cleaned_columns[biggest[0]]
    print(np.shape(cleaned_columns))
    zipped = create_cor_matrix(cleaned_columns)

[(0, 1), (2, 1), (3, 5), (6, 4), (7, 1), (12, 1), (15, 1), (17, 4), (18, 4), (19, 4)]
(19, 250000)
[(0, 1), (2, 1), (5, 3), (6, 1), (11, 1), (16, 3), (17, 3), (18, 3)]
(18, 250000)
[(0, 1), (2, 1), (5, 1), (10, 1), (15, 2), (16, 2), (17, 2)]
(17, 250000)
[(0, 1), (2, 1), (5, 1), (10, 1), (15, 1), (16, 1)]
(16, 250000)
[(9, 1), (4, 1), (14, 1), (15, 1)]
(15, 250000)
[(13, 1), (14, 1)]
(14, 250000)


In [11]:
zipped = create_cor_matrix(cleaned_columns_te)
while len(zipped) > 0 :
    biggest = (-1,-1)
    cnt = count(zipped)
    print(cnt)
    for c in cnt:
        if c[1] > biggest[1]:
            biggest = c
    del cleaned_columns_te[biggest[0]]
    print(np.shape(cleaned_columns_te))
    zipped = create_cor_matrix(cleaned_columns_te)

[(0, 1), (2, 1), (3, 5), (6, 4), (7, 1), (12, 1), (15, 1), (17, 4), (18, 4), (19, 4)]
(19, 568238)
[(0, 1), (2, 1), (5, 3), (6, 1), (11, 1), (16, 3), (17, 3), (18, 3)]
(18, 568238)
[(0, 1), (2, 1), (5, 1), (10, 1), (15, 2), (16, 2), (17, 2)]
(17, 568238)
[(0, 1), (2, 1), (5, 1), (10, 1), (15, 1), (16, 1)]
(16, 568238)
[(9, 1), (4, 1), (14, 1), (15, 1)]
(15, 568238)
[(13, 1), (14, 1)]
(14, 568238)


In [12]:
np.shape(cleaned_columns_te)

(14, 568238)

In [13]:

batch_size = 1
max_iters = 20
#gamma = 0.01
lambda_ = 0.01

In [14]:
gammas = [0.001, 0.002, 0.005, 0.01, 0.02, 0.03]

In [15]:
# standardize the data
std_data, mean, std = standardize(np.transpose(cleaned_columns))

In [16]:
std_data_te, mean_te, std_te = standardize(np.transpose(cleaned_columns_te))

In [17]:
# build the model (add 1's for the w0)
tx, y = build_model_data(std_data, yb)

In [18]:
tx_te, y_te = build_model_data(std_data_te, yb_te)

In [ ]:
print(tx.shape)

In [ ]:
initial_w = np.random.randint(0,1,tx.shape[1])

In [ ]:
from cross_validation import * 
degree = 3
k_fold = 4
lambdas = np.logspace(-4, 0, 30)
degrees = [3,5,7]
seed = 1
k_indices = build_k_indices(y, k_fold, seed)


In [76]:
initial_w = np.random.randint(0,1,len(tx.T))
max_iters = 30
gammas = np.arange(0.001,0.01,0.001)

In [ ]:
print(initial_w.shape)

In [ ]:
cross_validation_gradient_descent(y, tx, k_fold, initial_w=initial_w, max_iters = max_iters, gammas = gammas)

In [ ]:
cross_validation_stochastic_gradient_descent(y, tx, k_fold, initial_w=initial_w, max_iters = max_iters, gammas = gammas, batch_size = 2)

In [23]:
k_fold = 4
lambdas = np.logspace(-5, 0, 9)

In [25]:
from cross_validation import * 
_,_, acc = cross_validation_ridge_regression(y1, x1, k_fold, lambdas, 7)
print(acc)

NameError: name 'y1' is not defined

In [25]:
degrees = [1,2,3]

In [23]:
from build_polynomial import build_poly

In [70]:
best_lambda,best_rmse, acc, deg = cross_validation_ridge_regression(y, tx, k_fold, lambdas, degrees)

76.80720000000001


In [74]:
print(best_rmse)
print(deg)
print(best_lambda)

0.9631752942590047
7
0.0001


In [64]:
print(acc)
print(deg)
print(best_lambda)

76.80720000000001
7
0.0001


In [65]:
model_tr = build_poly(tx, deg)
model_te = build_poly(tx_te, deg)

In [66]:
w = ridge_regression(y, model_tr, best_lambda)

In [68]:
from hilp import *

In [69]:
y_pred = predict_y(model_te, w)

name = 'ridge_try'
create_csv_submission(ids_te, y_pred, name)

In [44]:

w = ridge_regression(y, model_tr, best_lambda)
y_pred=predict_y(model_te, w)
acc = compute_accuracy(y_pred, y_te)
print(acc)

10.672464706689803


In [59]:
cross_validation_logistic_regression(y, tx, k_fold, max_iters, gammas, degrees)

65.73320000000001


(0.005, 8.632454286050928, 65.73320000000001, 2)

In [39]:
cross_validation_reg_logistic_regression(y, tx, k_fold, max_iters, lambdas, gammas, degrees)

65.8896
65.6672
65.7552
65.6208
65.8896
65.6672
65.7552
65.6208
65.8896
65.6672
65.7552
65.6208
65.8896
65.6672
65.7552
65.6208
65.8896
65.6672


KeyboardInterrupt: 

In [35]:
from logistic_regression import *

In [36]:
tx.shape

(250000, 15)

In [37]:
initial_w = np.random.randint(0,1,len(tx.T))

In [38]:
reg_logistic_regression(y, tx, 0.01, initial_w, 30, 0.1)

([-30885.832075338363,
  -17194.338546267354,
  -10632.26182348934,
  -6953.48585265316,
  -4707.977008824804,
  -3263.448975141417,
  -2301.0394503240723,
  -1643.7523659692483,
  -1186.543570675488,
  -863.988999161175,
  -633.8592001964427,
  -468.1492902582386,
  -347.89556278819686,
  -260.0429819406038,
  -195.48367116642265,
  -147.79350536032845,
  -112.39903755353427,
  -86.0185981136331,
  -66.28091679461461,
  -51.46176424813003,
  -40.30051266970167,
  -31.87079102123262,
  -25.48860916982272,
  -20.646709483690266,
  -16.967200014084806,
  -14.167770925167666,
  -12.036395449145985,
  -10.413320334409104,
  -9.17816761751877,
  -8.239535871860651],
 [array([-0.1629328 , -0.15851702, -0.22727118,  0.07403441,  0.00977652,
          0.07680909,  0.0849063 , -0.04045015,  0.08302395,  0.08298306,
          0.08306983,  0.08285546, -0.0718799 ,  0.0830856 , -0.13583214]),
  array([-0.27667927, -0.28276703, -0.38054587,  0.12570854,  0.01817992,
          0.13026115,  0.1446831